In [2]:
# STEP 1: Install all the necessary libraries.
# LangChain for orchestration, LangChain-Groq for Groq LLM.
!pip install -q langchain langchain-groq langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 30.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.8/130.8 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.3 MB/s eta 0:00:00


In [3]:
# STEP 2: Gather your spellbooks.
from langchain_groq import ChatGroq
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder
)
from langchain_core.runnables import RunnableWithMessageHistory
from langchain_core.chat_history import (
    BaseChatMessageHistory,
    InMemoryChatMessageHistory
)
from langchain_core.output_parsers.xml import XMLOutputParser
from IPython.display import Markdown, display
from google.colab import userdata
import os

In [4]:
# STEP 3: Whisper your secret key to the winds.
try:
    api_key = userdata.get("GROQ_API_KEY")
except Exception:
    api_key = os.getenv("GROQ_API_KEY")
    if not api_key:
        raise ValueError("GROQ_API_KEY not found. Please set it!")

# STEP 3b: Summon the Groq-powered LLM.
llm = ChatGroq(
    model="llama-3.3-70b-versatile",
    api_key=api_key,
    temperature=0.3,
)

In [5]:
# STEP 4: 📜 Instruct your LLM to always output valid XML.
# -----------------------------------------------
# ⚡️ CRUCIAL: If you do not explicitly prompt your LLM to respond in valid XML,
# XMLOutputParser will fail because it expects well-formed XML!
# -----------------------------------------------

system_msg = SystemMessagePromptTemplate.from_template(
    "You are an XML generator assistant. Always respond in well-formed XML with <response> root tag."
)

human_msg = HumanMessagePromptTemplate.from_template(
    "{input}"
)

chat_prompt = ChatPromptTemplate.from_messages([
    system_msg,
    MessagesPlaceholder(variable_name="history"),
    human_msg
])

In [6]:
# STEP 5: Archive the echoes of your conversations.
store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

In [7]:
# STEP 6: 🗝️ The heart of this spell!
# -----------------------------------------------
# 💡 XMLOutputParser expects valid XML and parses it into ElementTree.
# It will raise an error if the LLM’s output is not valid XML.
# So your prompt must always guide the LLM carefully.
# -----------------------------------------------

chat_chain = chat_prompt | llm

# Add the XML parser
parsed_chain = chat_chain | XMLOutputParser()

# Do NOT attach XMLOutputParser inside the chain
chat_chain = chat_prompt | llm

# Wrap with memory support
chatbot = RunnableWithMessageHistory(
    chat_chain,  # << Just raw LLM
    get_session_history,
    input_messages_key="input",
    history_messages_key="history"
)

In [9]:
import xml.etree.ElementTree as ET

session_id = "chat-session-xml-001"

user_inputs = [
    "Describe the confusion matrix in an XML format with fields <answer> and <source>.",
    "Give an example of precision and recall wrapped in <answer> and <source>."
]

print(f"Starting chat loop with session ID: {session_id}")

for input_text in user_inputs:
    print(f"\nUser: {input_text}")
    raw_response = chatbot.invoke(
        {"input": input_text},
        config={"configurable": {"session_id": session_id}}
    )
    print("Raw Response:", raw_response)

    # ✅ FIX: Access the content string!
    response_tree = ET.fromstring(raw_response.content)

    answer = response_tree.find("answer").text if response_tree.find("answer") is not None else "No <answer> found"
    source = response_tree.find("source").text if response_tree.find("source") is not None else "No <source> found"
    display(Markdown(f"**Answer:** {answer}\n\n**Source:** {source}"))

print("\n--- Stored Chat History ---")
for message in store[session_id].messages:
    print(f"{message.type.capitalize()}: {message.content}")

Starting chat loop with session ID: chat-session-xml-001

User: Describe the confusion matrix in an XML format with fields <answer> and <source>.
Raw Response: content='<response>\n  <answer>\n    The confusion matrix is a table that is used to describe the performance of a classification model, such as a logistic regression or decision tree, against a test dataset. \n    The matrix itself is relatively simple to understand, but the related terminology can be confusing. \n    The matrix has the following structure:\n    - True Positives (TP): These are the correctly predicted positive values.\n    - True Negatives (TN): These are the correctly predicted negative values.\n    - False Positives (FP): These are the incorrectly predicted positive values.\n    - False Negatives (FN): These are the incorrectly predicted negative values.\n  </answer>\n  <source>Wikipedia, Machine Learning documentation, and various academic papers on classification models and model evaluation metrics.</source

**Answer:** 
    The confusion matrix is a table that is used to describe the performance of a classification model, such as a logistic regression or decision tree, against a test dataset. 
    The matrix itself is relatively simple to understand, but the related terminology can be confusing. 
    The matrix has the following structure:
    - True Positives (TP): These are the correctly predicted positive values.
    - True Negatives (TN): These are the correctly predicted negative values.
    - False Positives (FP): These are the incorrectly predicted positive values.
    - False Negatives (FN): These are the incorrectly predicted negative values.
  

**Source:** Wikipedia, Machine Learning documentation, and various academic papers on classification models and model evaluation metrics.


User: Give an example of precision and recall wrapped in <answer> and <source>.
Raw Response: content='<response>\n  <answer>\n    Precision and recall are two important metrics used to evaluate the performance of a classification model. \n    Precision is the ratio of True Positives (TP) to the sum of True Positives (TP) and False Positives (FP), i.e., Precision = TP / (TP + FP). \n    Recall is the ratio of True Positives (TP) to the sum of True Positives (TP) and False Negatives (FN), i.e., Recall = TP / (TP + FN).\n    For example, suppose we have a model that predicts whether a person has a disease or not, and we get the following results:\n    - True Positives (TP): 80 (correctly predicted to have the disease)\n    - True Negatives (TN): 700 (correctly predicted to not have the disease)\n    - False Positives (FP): 20 (incorrectly predicted to have the disease)\n    - False Negatives (FN): 10 (incorrectly predicted to not have the disease)\n    Then, Precision = 80 / (80 + 20) =

**Answer:** 
    Precision and recall are two important metrics used to evaluate the performance of a classification model. 
    Precision is the ratio of True Positives (TP) to the sum of True Positives (TP) and False Positives (FP), i.e., Precision = TP / (TP + FP). 
    Recall is the ratio of True Positives (TP) to the sum of True Positives (TP) and False Negatives (FN), i.e., Recall = TP / (TP + FN).
    For example, suppose we have a model that predicts whether a person has a disease or not, and we get the following results:
    - True Positives (TP): 80 (correctly predicted to have the disease)
    - True Negatives (TN): 700 (correctly predicted to not have the disease)
    - False Positives (FP): 20 (incorrectly predicted to have the disease)
    - False Negatives (FN): 10 (incorrectly predicted to not have the disease)
    Then, Precision = 80 / (80 + 20) = 0.8 and Recall = 80 / (80 + 10) = 0.889.
  

**Source:** Wikipedia, Machine Learning documentation, and various academic papers on classification models and model evaluation metrics, such as the scikit-learn documentation and the book "Pattern Recognition and Machine Learning" by Christopher Bishop.


--- Stored Chat History ---
Human: Describe the confusion matrix in an XML format with fields <answer> and <source>.
Ai: <response>
  <answer>The confusion matrix is a table used to evaluate the performance of a classification model. It has four main components: True Positives (TP), True Negatives (TN), False Positives (FP), and False Negatives (FN). The matrix is used to calculate various metrics such as accuracy, precision, recall, and F1 score, which help in understanding the model's performance.</answer>
  <source>The concept of the confusion matrix is widely used in machine learning and data science, and is often referenced in academic papers and online resources, including Wikipedia and scikit-learn documentation.</source>
</response>
Human: Describe the confusion matrix in an XML format with fields <answer> and <source>.
Ai: <response>
  <answer>
    The confusion matrix is a table that is used to describe the performance of a classification model, such as a logistic regression